# Simplicity

The safe language for smart contracts on bitcoin.

In [1]:
#program;;
#require "containers";;
module Fmt = CCFormat;;
#logic;;

/home/simon/.opam/4.03.0/lib/containers/monomorphic: added to search path
/home/simon/.opam/4.03.0/lib/containers/monomorphic/containers_monomorphic.cma: loaded
/home/simon/.opam/4.03.0/lib/result: added to search path
/home/simon/.opam/4.03.0/lib/result/result.cma: loaded
/home/simon/.opam/4.03.0/lib/uchar: added to search path
/home/simon/.opam/4.03.0/lib/containers: added to search path
/home/simon/.opam/4.03.0/lib/containers/containers.cma: loaded
module Fmt = CCFormat


## Basic types

In [2]:
type ty =
  | One
  | Prod of ty * ty
  | Sum of ty * ty
  
let one = One
let prod a b = Prod(a,b)
let sum a b = Sum (a,b)


(** a sequent, i.e. a pair of and input type and an output type *)
type top_ty = ty * ty


type ty = One | Prod of ty * ty | Sum of ty * ty
val one : ty = One
val prod : ty -> ty -> ty = <fun>
val sum : ty -> ty -> ty = <fun>
type top_ty = ty * ty


In [3]:
#program;;

let rec pp_ty out = function
  | One -> Fmt.string out "1"
  | Prod (a,b) -> Fmt.fprintf out "(@[%a @<1>×@ %a@])" pp_ty a pp_ty b
  | Sum (a,b) -> Fmt.fprintf out "(@[%a +@ %a@])" pp_ty a pp_ty b
;;
#install_printer pp_ty;;

#logic;;

val pp_ty : Fmt.t -> ty -> unit = <fun>


In [4]:

(** Simplicity expressions *)
type expr =
  | Iden
  | Unit
  | Inj_l of expr
  | Inj_r of expr
  | Comp of ty * expr * expr  (* intermediate type needs to be provided *)
  | Case of expr * expr
  | Pair of expr * expr
  | Take of expr
  | Drop of expr
  
(* constructors *)

let iden = Iden [@@macro]
let unit = Unit [@@macro]
let inj_l x = Inj_l x [@@macro]
let inj_r x = Inj_r x [@@macro]
let comp ty a b = Comp(ty,a,b) [@@macro]
let case a b = Case(a,b) [@@macro]
let pair a b = Pair(a,b) [@@macro]
let take s = Take s [@@macro]
let drop s = Drop s [@@macro]

type expr =
    Iden
  | Unit
  | Inj_l of expr
  | Inj_r of expr
  | Comp of ty * expr * expr
  | Case of expr * expr
  | Pair of expr * expr
  | Take of expr
  | Drop of expr
val iden : expr = Iden
val unit : expr = Unit
val inj_l : expr -> expr = <fun>
val inj_r : expr -> expr = <fun>
val comp : ty -> expr -> expr -> expr = <fun>
val case : expr -> expr -> expr = <fun>
val pair : expr -> expr -> expr = <fun>
val take : expr -> expr = <fun>
val drop : expr -> expr = <fun>


In [5]:
#program;;


let rec pp_expr out = function
  | Iden -> Fmt.string out "iden"
  | Unit -> Fmt.string out "unit"
  | Comp (_,a,b) -> Fmt.fprintf out "(@[comp@ %a@ %a@])" pp_expr a pp_expr b
  | Inj_l a -> Fmt.fprintf out "(@[inj_l@ %a]@])" pp_expr a
  | Inj_r a -> Fmt.fprintf out "(@[inj_r@ %a]@])" pp_expr a
  | Take a -> Fmt.fprintf out "(@[take@ %a]@])" pp_expr a
  | Drop a -> Fmt.fprintf out "(@[drop@ %a]@])" pp_expr a
  | Case (a,b) -> Fmt.fprintf out "(@[<hv1>case@ %a@ %a@])" pp_expr a pp_expr b
  | Pair (a,b) -> Fmt.fprintf out "(@[<hv1>pair@ %a@ %a@])" pp_expr a pp_expr b
;;

#install_printer pp_expr;;

#logic;;

val pp_expr : Fmt.t -> expr -> unit = <fun>


## Typing

We could not write a mere typechecking function with the paper expressions, because the typing rule for `Comp` is a "cut". Instead, we ask the user to provide this middle type in `Comp` explicitly.

In [6]:

(** Predicate: [expr_ty e ty = true] means that [e : ty] holds *)
let rec expr_ty (e:expr) (ty:top_ty) : bool =
  match e, ty with
  | Iden, (a, b) -> a = b
  | Unit, (_, One) -> true
  | Comp (b,s,t), (a, c) ->
    expr_ty s (a,b) && expr_ty t (b,c)
  | Case (s,t), (Prod(Sum (a,b),c), d) ->
    expr_ty s (prod a c, d) && expr_ty t (prod b c, d)
  | Pair(s,t), (a,Prod(b,c)) ->
    expr_ty s (a, b) && expr_ty t (a, c)
  | Take t, (Prod(a,b),c) -> expr_ty t (a,c)
  | Drop t, (Prod(a,b),c) -> expr_ty t (b,c)
  | Inj_l t, (a, Sum(b,c)) -> expr_ty t (a,b)
  | Inj_r t, (a, Sum(b,c)) -> expr_ty t (a,c)
  | Unit, _ | Case _, _ | Take _, _ | Drop _, _ 
  | Pair _, _ | Inj_l _, _ | Inj_r _, _ 
    -> false
;;



val expr_ty : expr -> top_ty -> bool = <fun>


We might think that typing of an `expr` is unique, but it is not the case:

In [7]:

verify
  (fun e ty1 ty2 ->
     expr_ty e ty1 && expr_ty e ty2 ==> ty1 = ty2)


- : expr -> top_ty -> top_ty -> bool = <fun>
2 base ground recursive instances
Counterexample (after 2 steps, 0.00s):
 let e = Unit
 let ty1 = (Prod (Prod (Prod (One, One), One), One), One)
 let ty2 = (Prod (Prod (One, One), One), One)
module CX : sig val e : expr val ty1 : ty * ty val ty2 : ty * ty end
[✗] Conjecture refuted.


expr,(expr_ty_1274 |sko:e_1557| |sko:ty2_1559|)
expansions,prod
expr,(expr_ty_1274 |sko:e_1557| |sko:ty1_1558|)
expansions,prod
variable,value
(ty2 : `(ty * ty)`),"(Prod(Prod(One, One), One), One)"
(ty1 : `(ty * ty)`),"(Prod(Prod(Prod(One, One), One), One), One)"
(e : expr),Unit


In [24]:

verify
  (fun e ty1 ty2 ->
     e <> Unit && e <> Iden ==>
     expr_ty e ty1 && expr_ty e ty2 ==> ty1 = ty2)


- : expr -> top_ty -> top_ty -> bool = <fun>
2 base ground recursive instances
Counterexample (after 7 steps, 0.03s):
 let e = Comp (One, Unit, Unit)
 let ty1 = (Prod (Sum (Prod (One, One), Prod (One, One)), Prod (One, One)), One)
 let ty2 = (Prod (Sum (One, One), One), One)
module CX : sig val e : expr val ty1 : ty * ty val ty2 : ty * ty end
[✗] Conjecture refuted.


expr,(expr_ty_1274 |sko:e_1902| |sko:ty1_1903|)
expansions,prod
expr,(expr_ty_1274 |sko:e_1902| |sko:ty2_1904|)
expansions,prod
expr,(expr_ty_1274 (Comp_1249_ARG_1_1354 |sko:e_1902|) (|tuple_mk.`(ty * ty)`_1362_1363| (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty1_1903|) (Comp_1249_ARG_0_1353 |sko:e_1902|)))
expansions,prod
expr,(expr_ty_1274 (Comp_1249_ARG_1_1354 |sko:e_1902|) (|tuple_mk.`(ty * ty)`_1362_1363| (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty2_1904|) (Comp_1249_ARG_0_1353 |sko:e_1902|)))
expansions,prod
expr,(expr_ty_1274 (Comp_1249_ARG_2_1355 |sko:e_1902|) (|tuple_mk.`(ty * ty)`_1362_1363| (Comp_1249_ARG_0_1353 |sko:e_1902|) (|tuple_get.`(ty * ty)`_1362.1_1365| |sko:ty1_1903|)))
expansions,prod
expr,(let ((a!1 (Prod_1234 (Sum_1235_ARG_0_1351 (Prod_1234_ARG_0_1349 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty1_1903|))) (Prod_1234_ARG_1_1350 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty1_1903|))))) (expr_ty_1274 (Case_1250_ARG_0_1356 |sko:e_1902|) (|tuple_mk.`(ty * ty)`_1362_1363| a!1 (|tuple_get.`(ty * ty)`_1362.1_1365| |sko:ty1_1903|))))


## Evaluation


Evaluation reduces a term into a value. Values, like expressions, are typed.

In [9]:
(* first, we need values *)

(** Values obtained when evaluating Simplicity expressions *)
type value =
    | V_unit
    | V_left of value
    | V_right of value
    | V_pair of value * value
    
let v_unit = V_unit [@@macro]
let v_left x = V_left x [@@macro]
let v_right x = V_right x [@@macro]
let v_pair x y = V_pair (x,y) [@@macro]


type value =
    V_unit
  | V_left of value
  | V_right of value
  | V_pair of value * value
val v_unit : value = V_unit
val v_left : value -> value = <fun>
val v_right : value -> value = <fun>
val v_pair : value -> value -> value = <fun>


In [10]:
#program;;

let rec pp_value out = function
  | V_unit -> Format.fprintf out "()"
  | V_left v -> Format.fprintf out "@<1>σ0%a" pp_value v
  | V_right v -> Format.fprintf out "@<1>σ1%a" pp_value v
  | V_pair (v1,v2) -> Format.fprintf out "(@[%a,@ %a@])" pp_value v1 pp_value v2
;;

#install_printer pp_value ;;

#logic ;;

val pp_value : Format.formatter -> value -> unit = <fun>


In [11]:

let rec value_ty (v:value) (ty:ty) : bool =
  match v, ty with
  | V_unit, One -> true
  | V_pair (v1,v2), Prod(a,b) -> value_ty v1 a && value_ty v2 b
  | V_left v', Sum (a,_) -> value_ty v' a
  | V_right v', Sum (_,b) -> value_ty v' b
  | V_unit, _ | V_pair _, _ | V_left _, _ | V_right _, _ -> false


val value_ty : value -> ty -> bool = <fun>


Evaluation takes an expression of type $ A \vdash B $, and a value of type $A$, and
returns a value of type $B$.

It always terminates and takes predictable time and space.

In [35]:

(** Main evaluation function. It is partial because of type errors *)
let rec eval (e:expr) (v:value) : value option =
    let open Option in
    match e, v with
    | Unit, _ -> Some V_unit
    | Comp (_,s,t), _ ->
      eval s v >>= fun v' -> eval t v'
    | Iden, _ -> Some v
    | Inj_l t, _ -> eval t v >|= v_left
    | Inj_r t, _ -> eval t v >|= v_right
     (*
    | Case (s, t), V_pair (V_left a, c) ->
      eval s (v_pair a c)
    | Case (s, t), V_pair (V_right b, c) ->
      eval t (v_pair b c)
   
    | Pair (s, t), _ ->
      eval s v >>= fun v1 ->
      eval t v >|= fun v2 ->
      v_pair v1 v2 
    | Take t, V_pair (a,_) ->
      eval t a 
    | Drop t, V_pair (_,b) ->
      eval t b  *)
    | _ ->
      None  (* ill typed *)
[@@adm 0n]

val eval : expr -> value -> value option = <fun>


### Example

We can define the type of individual bits: $\mathbb{2} \equiv \mathbb{1} + \mathbb{1}$

In [13]:
let bit : ty = sum one one

let b_zero = v_left v_unit
let b_one  = v_right v_unit
;;

value_ty b_zero bit ;;
value_ty b_one  bit ;;

val bit : ty = (1 + 1)
val b_zero : value = σ0()
val b_one : value = σ1()
- : bool = true
- : bool = true


In [14]:
42 ;;

- : Z.t = 42


In [15]:
1+1;;

- : Z.t = 2


In [16]:
(* bit negation *)

let bit_not : expr =
    comp (prod bit one) (pair iden unit) (case (inj_r unit) (inj_l unit))
;;

b_zero, eval bit_not b_zero ;;
b_one,  eval bit_not b_one ;;


expr_ty bit_not (bit, bit) ;;

val bit_not : expr =
  (comp (pair iden unit) (case (inj_r unit]) (inj_l unit])))
- : value * '_a = (σ0(), <poly>)
- : value * '_a = (σ1(), <poly>)
- : bool = true


In [25]:

(* how to infer the intermediate type of `bit_not` *)

verify (fun ty ->
  let bit_not = comp ty (pair iden unit) (case (inj_r unit) (inj_l unit)) in
  not (expr_ty bit_not (bit,bit)))
;;
  
CX.ty;;

(* check result *)
let bit_not' =
  comp CX.ty (pair iden unit) (case (inj_r unit) (inj_l unit))
in
not (expr_ty bit_not' (bit,bit));;

CX.ty = prod bit one;;

- : ty -> bool = <fun>
1 base ground recursive instance
Counterexample (after 10 steps, 0.02s):
 let ty = Prod (Sum (One, One), One)
module CX : sig val ty : ty end
[✗] Conjecture refuted.
- : ty = ((1 + 1) × 1)
- : bool = false
- : bool = true


expr,(expr_ty_1274 (Comp_1249 |sko:ty_1923| (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246))) (|tuple_mk.`(ty * ty)`_1362_1363| (Sum_1235 One_1233 One_1233) (Sum_1235 One_1233 One_1233)))
expansions,prod
expr,(expr_ty_1274 (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246)) (|tuple_mk.`(ty * ty)`_1362_1363| |sko:ty_1923| (Sum_1235 One_1233 One_1233)))
expansions,prod
expr,(expr_ty_1274 (Pair_1251 Iden_1245 Unit_1246) (|tuple_mk.`(ty * ty)`_1362_1363| (Sum_1235 One_1233 One_1233) |sko:ty_1923|))
expansions,prod
expr,(let ((a!1 (|tuple_mk.`(ty * ty)`_1362_1363| (Prod_1234 (Sum_1235_ARG_0_1351 (Prod_1234_ARG_0_1349 |sko:ty_1923|)) (Prod_1234_ARG_1_1350 |sko:ty_1923|)) (Sum_1235 One_1233 One_1233)))) (expr_ty_1274 (Inj_r_1248 Unit_1246) a!1))
expansions,prod
expr,(let ((a!1 (|tuple_mk.`(ty * ty)`_1362_1363| (Prod_1234 (Sum_1235_ARG_0_1351 (Prod_1234_ARG_0_1349 |sko:ty_1923|)) (Prod_1234_ARG_1_1350 |sko:ty_1923|)) One_1233))) (expr_ty_1274 Unit_1246 a!1))
expansions,prod
expr,(let ((a!1 (Case_1250_ARG_0_1356 (Comp_1249 |sko:ty_1923| (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246))))) (a!2 (Prod_1234 (Sum_1235_ARG_0_1351 (Prod_1234_ARG_0_1349 (Sum_1235 One_1233 One_1233))) (Prod_1234_ARG_1_1350 (Sum_1235 One_1233 One_1233))))) (expr_ty_1274 a!1 (|tuple_mk.`(ty * ty)`_1362_1363| a!2 (Sum_1235 One_1233 One_1233))))


In [26]:
let goal = fun ty ->
  let bit_not = comp ty (pair iden unit) (case (inj_r unit) (inj_l unit)) in
  not (expr_ty bit_not (bit,bit))
;;

Verify.top "goal";;

val goal : ty -> bool = <fun>
1 base ground recursive instance
Counterexample (after 10 steps, 0.02s):
 let ty = Prod (Sum (One, One), One)
module CX : sig val ty : ty end
- : Imandra_lib.Kernel.Verify.t =
Imandra_lib.Kernel.Verify.Refuted (Some <abstr>, Some <abstr>)


expr,(expr_ty_1274 (Comp_1249 |sko:ty_1938| (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246))) (|tuple_mk.`(ty * ty)`_1362_1363| (Sum_1235 One_1233 One_1233) (Sum_1235 One_1233 One_1233)))
expansions,prod
expr,(expr_ty_1274 (Pair_1251 Iden_1245 Unit_1246) (|tuple_mk.`(ty * ty)`_1362_1363| (Sum_1235 One_1233 One_1233) |sko:ty_1938|))
expansions,prod
expr,(expr_ty_1274 Iden_1245 (|tuple_mk.`(ty * ty)`_1362_1363| (Sum_1235 One_1233 One_1233) (Prod_1234_ARG_0_1349 |sko:ty_1938|)))
expansions,prod
expr,(expr_ty_1274 Unit_1246 (|tuple_mk.`(ty * ty)`_1362_1363| (Sum_1235 One_1233 One_1233) (Prod_1234_ARG_1_1350 |sko:ty_1938|)))
expansions,prod
expr,(expr_ty_1274 (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246)) (|tuple_mk.`(ty * ty)`_1362_1363| |sko:ty_1938| (Sum_1235 One_1233 One_1233)))
expansions,prod
expr,(let ((a!1 (Case_1250_ARG_0_1356 (Comp_1249 |sko:ty_1938| (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246))))) (a!2 (Prod_1234 (Sum_1235_ARG_0_1351 (Prod_1234_ARG_0_1349 (Sum_1235 One_1233 One_1233))) (Prod_1234_ARG_1_1350 (Sum_1235 One_1233 One_1233))))) (expr_ty_1274 a!1 (|tuple_mk.`(ty * ty)`_1362_1363| a!2 (Sum_1235 One_1233 One_1233))))


## Well Typed Expressions

Any well typed expressions evaluates to a value successfully.

In [19]:

theorem eval_well_typed (e:expr) (a, b : ty * ty) (v:value) =
  expr_ty e (a,b) &&
  value_ty v a
  ==>
  begin match eval e v with
    | None -> false
    | Some v' -> value_ty v' b
  end
[@@induct e]
;;


val eval_well_typed : expr -> ty * ty -> value -> bool = <fun>
File "_none_", line 1:
Error: RT.mk_mono_app: eval should be a defined fun


### Example: half adder

In [20]:
let two_two = prod bit bit 


let half_adder =
  case (drop (pair (inj_l unit) iden))
       (drop (pair iden bit_not))


val two_two : ty = ((1 + 1) × (1 + 1))
val half_adder : expr =
  (case
    (drop (pair (inj_l unit]) iden)])
    (drop
     (pair iden (comp (pair iden unit) (case (inj_r unit]) (inj_l unit]))))]))


In [21]:
expr_ty half_adder (prod bit bit, two_two);;

- : bool = true


In [22]:
(* TODO: use `instances` to find the type of `half_adder` *)

verify (fun ty -> not @@ expr_ty half_adder ty);;

- : top_ty -> bool = <fun>
1 base ground recursive instance
Counterexample (after 22 steps, 0.08s):
 let ty = (Prod (Sum (One, One), Sum (One, One)), Prod (Sum (One, One), Sum (One, One)))
module CX : sig val ty : ty * ty end
[✗] Conjecture refuted.


expr,(let ((a!1 (Pair_1251 Iden_1245 (Comp_1249 (Prod_1234 (Sum_1235 One_1233 One_1233) One_1233) (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246)))))) (let ((a!2 (Case_1250 (Drop_1253 (Pair_1251 (Inj_l_1247 Unit_1246) Iden_1245)) (Drop_1253 a!1)))) (expr_ty_1274 a!2 |sko:ty_1874|)))
expansions,prod
expr,(let ((a!1 (Prod_1234 (Sum_1235_ARG_0_1351 (Prod_1234_ARG_0_1349 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty_1874|))) (Prod_1234_ARG_1_1350 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty_1874|))))) (expr_ty_1274 (Drop_1253 (Pair_1251 (Inj_l_1247 Unit_1246) Iden_1245)) (|tuple_mk.`(ty * ty)`_1362_1363| a!1 (|tuple_get.`(ty * ty)`_1362.1_1365| |sko:ty_1874|))))
expansions,prod
expr,(let ((a!1 (Pair_1251 Iden_1245 (Comp_1249 (Prod_1234 (Sum_1235 One_1233 One_1233) One_1233) (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246))))) (a!2 (Prod_1234 (Sum_1235_ARG_1_1352 (Prod_1234_ARG_0_1349 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty_1874|))) (Prod_1234_ARG_1_1350 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty_1874|))))) (expr_ty_1274 (Drop_1253 a!1) (|tuple_mk.`(ty * ty)`_1362_1363| a!2 (|tuple_get.`(ty * ty)`_1362.1_1365| |sko:ty_1874|))))
expansions,prod
expr,(let ((a!1 (Pair_1251 Iden_1245 (Comp_1249 (Prod_1234 (Sum_1235 One_1233 One_1233) One_1233) (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246)))))) (expr_ty_1274 a!1 (|tuple_mk.`(ty * ty)`_1362_1363| (Prod_1234_ARG_1_1350 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty_1874|)) (|tuple_get.`(ty * ty)`_1362.1_1365| |sko:ty_1874|))))
expansions,prod
expr,(expr_ty_1274 Iden_1245 (|tuple_mk.`(ty * ty)`_1362_1363| (Prod_1234_ARG_1_1350 (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty_1874|)) (Prod_1234_ARG_0_1349 (|tuple_get.`(ty * ty)`_1362.1_1365| |sko:ty_1874|))))
expansions,prod
expr,(let ((a!1 (Pair_1251 Iden_1245 (Comp_1249 (Prod_1234 (Sum_1235 One_1233 One_1233) One_1233) (Pair_1251 Iden_1245 Unit_1246) (Case_1250 (Inj_r_1248 Unit_1246) (Inj_l_1247 Unit_1246)))))) (let ((a!2 (Case_1250 (Drop_1253 (Pair_1251 (Inj_l_1247 Unit_1246) Iden_1245)) (Drop_1253 a!1)))) (expr_ty_1274 (Comp_1249_ARG_1_1354 a!2) (|tuple_mk.`(ty * ty)`_1362_1363| (|tuple_get.`(ty * ty)`_1362.0_1364| |sko:ty_1874|) (Comp_1249_ARG_0_1353 a!2)))))


In [23]:
#h List.map

doc,Map a function over a list.map f [] = []map f [x] = [f x]map f (x :: tail) = f x :: map f tail
iml,(fun (f : 'b -> 'a) -> (fun (l : 'b list) -> if (l = []) then [] else let (x : 'b) = List.hd(l) and (l2 : 'b list) = List.tl(l) in ((f x) :: (List.map f l2))))
name,List.map
recursive,true
Measured subset,[1]
parametric,true
location,Characters 12225-12316: #h List.map
